In [ ]:
import sys
sys.path.extend([
    '../../bach',
    '../'
])

from sql_models.graph_operations import get_node
from sql_models.model import CustomSqlModel, Materialization
from sql_models.sql_generator import to_sql
from sql_models.cli_util import print_graph_info

from objectiv_bach.util import duplo_basic_features

import sqlalchemy

In [ ]:
engine = sqlalchemy.create_engine('postgresql://objectiv:@localhost:5432/objectiv')
def run_query(sql):
    with engine.connect() as conn:
        result = conn.execute(sql)
        return list(result)

In [ ]:
basic_features = duplo_basic_features()
# Show graph
print_graph_info(basic_features)

In [ ]:
#print(to_sql(basic_features)[:100])
print(to_sql(basic_features))

In [ ]:
rows = run_query(to_sql(basic_features))
print(len(rows))

In [ ]:
sub_graph = get_node(basic_features, ('feature_table',))
print_graph_info(sub_graph)
print('\n\n')
print(to_sql(sub_graph))


In [ ]:
custom_model_builder = CustomSqlModel(
    sql='''
    select *
    from {{basic}}
    limit {limit}
    '''
)
custom_model = custom_model_builder(limit=5, basic=basic_features)
print_graph_info(custom_model)
rows = run_query(to_sql(custom_model))
print(len(rows))

In [ ]:
modified_graph1 = custom_model.set(
    reference_path=('basic', 'sessionized_data'),
    session_gap_seconds=5
)
modified_graph2 = custom_model.set(
    reference_path=(),
    limit=10
)


In [ ]:
print_graph_info(modified_graph1)

In [ ]:
print_graph_info(modified_graph2)

In [ ]:
modified_graph_combined = modified_graph1\
.set(
    reference_path=(),
    limit=1337
).set_materialization(
    reference_path=('basic', 'sessionized_data'),
    materialization=Materialization.VIEW
)

# note: Materialization.VIEW does not yet have any effect

In [ ]:
print_graph_info(modified_graph_combined)

